# Thistlethwaite's Algorithm - Detailed Analysis

**Author:** Alex Toska  
**Affiliation:** University of Patras  
**Algorithm:** Thistlethwaite (1981)  

---

## Overview

Morwen Thistlethwaite's algorithm solves the Rubik's Cube using group-theoretic reduction through 4 phases, reducing the group size at each phase.

This notebook provides:
- Mathematical foundation and group theory
- Implementation details
- Step-by-step examples
- Performance analysis
- Comparison with other algorithms

## Setup

Import required modules and set up visualization.

In [ ]:
import sys
from pathlib import Path
import time
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Add src to path
sys.path.insert(0, str(Path.cwd().parent))

from src.cube.rubik_cube import RubikCube
from src.thistlethwaite.solver import ThistlethwaiteSolver
from src.cube.visualization import display_cube_unfolded
from src.cube.visualize_3d import visualize_cube_3d

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 5)

print("✓ Imports successful!")

## 1. Group Theory Foundation

Thistlethwaite's brilliant insight was to reduce the full Rubik's Cube group G₀ through increasingly restricted subgroups:

### The Four Groups

**G₀ → G₁ → G₂ → G₃ → G₄ (solved)**

| Group | Description | Size | Constraint |
|-------|-------------|------|------------|
| **G₀** | All possible states | 4.3×10¹⁹ | None |
| **G₁** | All edges oriented | 2.1×10¹⁸ | Edge orientation fixed |
| **G₂** | Edges in UD-slice + corners oriented | 1.9×10¹⁶ | Middle layer edges in place |
| **G₃** | Pieces in tetrads | 6.6×10⁸ | Corner and edge positions grouped |
| **G₄** | Solved state | 1 | All pieces solved |

Each phase reduces the state space by restricting moves and achieving specific cube properties.

## 2. Move Restrictions by Phase

### Phase 0 (G₀ → G₁): Orient all edges
**Allowed moves:** All 18 moves  
`U, U2, U', D, D2, D', F, F2, F', B, B2, B', L, L2, L', R, R2, R'`

**Goal:** Get all 12 edges properly oriented (no edge flips needed)

### Phase 1 (G₁ → G₂): Position middle layer edges + orient corners
**Allowed moves:** 14 moves (no F, F', B, B')  
`U, U2, U', D, D2, D', F2, B2, L, L2, L', R, R2, R'`

**Goal:** Get FR, FL, BR, BL edges into the middle layer

### Phase 2 (G₂ → G₃): Position corners and edges into tetrads
**Allowed moves:** 10 moves (only 90° on U/D, 180° on F/B/L/R)  
`U, U2, U', D, D2, D', F2, B2, L2, R2`

**Goal:** Group pieces into solvable tetrad positions

### Phase 3 (G₃ → G₄): Solve the cube
**Allowed moves:** 6 moves (only 180° turns)  
`U2, D2, F2, B2, L2, R2`

**Goal:** Fully solve the cube

## 3. Step-by-Step Example

Let's solve a scrambled cube and analyze each phase.

In [ ]:
# Create and scramble a cube
cube = RubikCube()
scramble = cube.scramble(moves=15, seed=42)

print("=== SCRAMBLE ===")
print(f"Scramble: {' '.join(scramble)}")
print(f"Scramble length: {len(scramble)} moves\n")

# Visualize scrambled cube
fig = visualize_cube_3d(cube, view_angles=(30, 45))
fig.suptitle("Scrambled Cube (15 moves)", fontsize=14, fontweight='bold')
plt.show()

print("\nCube state (2D unfolded):")
print(display_cube_unfolded(cube))

In [ ]:
# Solve using Thistlethwaite
solver = ThistlethwaiteSolver(use_pattern_databases=False)

print("=== SOLVING ===")
start_time = time.time()
solution = solver.solve(cube)
end_time = time.time()

print(f"\n✓ Solution found!")
print(f"Solution: {' '.join(solution)}")
print(f"Solution length: {len(solution)} moves")
print(f"Time: {end_time - start_time:.4f} seconds")

# Verify solution
test_cube = cube.copy()
for move in solution:
    test_cube.apply_move(move)

print(f"\nVerification: Cube is {'SOLVED ✓' if test_cube.is_solved() else 'NOT SOLVED ✗'}")

In [ ]:
# Visualize solved cube
fig = visualize_cube_3d(test_cube, view_angles=(30, 45))
fig.suptitle("Solved Cube", fontsize=14, fontweight='bold')
plt.show()

## 4. Phase-by-Phase Analysis

The solution moves can be analyzed to understand how each phase contributes to solving the cube.

### Phase Distribution

Typically:
- **Phase 0 (G₀→G₁):** 7-10 moves - Orient edges
- **Phase 1 (G₁→G₂):** 8-13 moves - Position middle edges, orient corners
- **Phase 2 (G₂→G₃):** 10-15 moves - Create tetrads
- **Phase 3 (G₃→G₄):** 5-14 moves - Final solve

**Total:** 30-52 moves (worst case: 52, typical: 35-45)

## 5. Performance Analysis

Let's benchmark the algorithm on multiple scrambles.

In [ ]:
# Benchmark on 10 scrambles
print("Running benchmark on 10 scrambles...\n")

results = []
for seed in range(42, 52):
    cube = RubikCube()
    scramble = cube.scramble(moves=20, seed=seed)
    
    solver = ThistlethwaiteSolver(use_pattern_databases=False)
    start = time.time()
    solution = solver.solve(cube)
    elapsed = time.time() - start
    
    results.append({
        'seed': seed,
        'scramble_length': len(scramble),
        'solution_length': len(solution),
        'time_seconds': elapsed
    })
    
    print(f"Seed {seed}: {len(solution)} moves in {elapsed:.3f}s")

# Create DataFrame
df = pd.DataFrame(results)

print("\n=== Summary Statistics ===")
print(f"Average solution length: {df['solution_length'].mean():.1f} moves")
print(f"Min solution length: {df['solution_length'].min()} moves")
print(f"Max solution length: {df['solution_length'].max()} moves")
print(f"Average time: {df['time_seconds'].mean():.4f} seconds")
print(f"Min time: {df['time_seconds'].min():.4f} seconds")
print(f"Max time: {df['time_seconds'].max():.4f} seconds")

In [ ]:
# Visualize results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Solution lengths
ax1.bar(df['seed'] - 42, df['solution_length'], color='#3498db', alpha=0.7)
ax1.axhline(y=df['solution_length'].mean(), color='red', linestyle='--', 
            label=f"Mean: {df['solution_length'].mean():.1f}")
ax1.set_xlabel('Test Case', fontsize=12)
ax1.set_ylabel('Solution Length (moves)', fontsize=12)
ax1.set_title('Thistlethwaite Solution Lengths', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# Execution times
ax2.bar(df['seed'] - 42, df['time_seconds'], color='#2ecc71', alpha=0.7)
ax2.axhline(y=df['time_seconds'].mean(), color='red', linestyle='--',
            label=f"Mean: {df['time_seconds'].mean():.3f}s")
ax2.set_xlabel('Test Case', fontsize=12)
ax2.set_ylabel('Time (seconds)', fontsize=12)
ax2.set_title('Thistlethwaite Execution Time', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Complexity Analysis

### Time Complexity

- **Phase 0:** O(2¹² × 18) = O(73,728) - Edge orientation
- **Phase 1:** O(2¹² × C(12,4) × 3⁸ × 14) = O(~10⁹) - Edge position + corner orientation
- **Phase 2:** O(8! × 8 × 10) = O(~3×10⁶) - Tetrad positioning
- **Phase 3:** O(8! × 12!/4! × 6) = O(~10¹⁰) - Final solve

**Overall:** O(10¹⁰) worst case, but practically very fast due to pruning

### Space Complexity

- **Pattern databases:** ~2 MB
- **Move tables:** ~1 MB
- **Search state:** Minimal (BFS/DFS depth-limited)

**Total:** ~3-5 MB

### Solution Length

- **Theoretical maximum:** 52 moves
- **Typical:** 35-45 moves
- **Best case:** ~30 moves

Compare with God's Number (20 moves optimal)

## 7. Strengths & Weaknesses

### ✅ Strengths

1. **Fast execution:** Typically solves in 0.1-2 seconds
2. **Guaranteed solution:** Always finds a solution
3. **Elegant mathematics:** Beautiful application of group theory
4. **Low memory:** Only ~2-5 MB required
5. **Educational value:** Great for understanding cube structure
6. **Predictable behavior:** Consistent performance

### ❌ Weaknesses

1. **Sub-optimal solutions:** 30-52 moves vs 20 optimal
2. **Complex implementation:** More intricate than simple layer-by-layer
3. **Pattern databases needed:** Requires pre-computation
4. **No move optimization:** Each phase independent
5. **Not competition-level:** Solutions too long for speed solving

### 🎯 Best Use Cases

- **Educational purposes** - Teaching group theory
- **Quick demonstrations** - Fast solving for demos
- **Historical importance** - First sub-minute algorithm
- **Research foundation** - Basis for later algorithms (Kociemba)
- **When speed > optimality** - Need fast results, don't care about move count

## 8. Comparison with Other Algorithms

### Thistlethwaite vs Kociemba vs Korf

| Feature | Thistlethwaite | Kociemba | Korf |
|---------|----------------|----------|------|
| **Year** | 1981 | 1992 | 1997 |
| **Phases** | 4 | 2 | 1 (IDA*) |
| **Solution Length** | 30-52 moves | <19 moves | ≤20 moves |
| **Speed** | ⚡ Very Fast (0.1-2s) | 🚀 Medium (0.001-5s) | 🐢 Variable (1-30s) |
| **Memory** | 💾 Low (~2 MB) | 💾 Medium (~80 MB) | 💾 Moderate (~45 MB) |
| **Optimality** | ❌ Sub-optimal | ⚠️ Near-optimal | ✅ Optimal |
| **Complexity** | Medium | High | Very High |
| **Best For** | Education, demos | Practical solving | Research, optimal |

### When to Use Thistlethwaite

Choose Thistlethwaite when:
- You need fast results (< 2 seconds)
- Solution length is not critical
- You want to understand group theory
- Memory is limited
- Teaching/learning algorithm design

## 9. Implementation Details

### Key Components

The implementation in this project includes:

1. **Coordinate Systems** (`src/thistlethwaite/coordinates.py`)
   - Edge orientation coordinate
   - Edge permutation coordinate
   - Corner orientation coordinate
   - Corner permutation coordinate

2. **Move Tables** (`src/thistlethwaite/move_tables.py`)
   - Pre-computed transitions for each coordinate
   - Separate tables for each phase

3. **Solver** (`src/thistlethwaite/solver.py`)
   - BFS/IDA* search for each phase
   - Phase transition logic
   - Solution concatenation

4. **Pattern Databases** (optional)
   - Pre-computed distance tables
   - Faster lookups during search

### Usage Example

```python
from src.cube.rubik_cube import RubikCube
from src.thistlethwaite.solver import ThistlethwaiteSolver

# Create scrambled cube
cube = RubikCube()
cube.scramble(moves=20, seed=42)

# Solve
solver = ThistlethwaiteSolver()
solution = solver.solve(cube)

# Apply solution
for move in solution:
    cube.apply_move(move)

assert cube.is_solved()
```

## 10. Exercises

Try these experiments:

### Exercise 1: Scramble Depth
Test how scramble depth affects solution length:
```python
for depth in [5, 10, 15, 20]:
    cube = RubikCube()
    cube.scramble(moves=depth, seed=42)
    solver = ThistlethwaiteSolver()
    solution = solver.solve(cube)
    print(f"Depth {depth}: {len(solution)} moves")
```

### Exercise 2: Pattern Databases
Compare performance with and without pattern databases:
```python
solver_no_db = ThistlethwaiteSolver(use_pattern_databases=False)
solver_with_db = ThistlethwaiteSolver(use_pattern_databases=True)
# Compare times...
```

### Exercise 3: Phase Analysis
Modify the solver to report moves per phase:
```python
# Add phase boundaries to solution output
# Analyze which phases use most moves
```

### Exercise 4: Custom Scrambles
Test specific scramble patterns:
```python
# Try super-flip (requires 20 moves optimal)
# Try checker pattern
# Try specific algorithms (Sune, T-perm, etc.)
```

## 11. References

### Original Papers

1. **Thistlethwaite, M. (1981)** - "A new approach to solving Rubik's Cube"
   - Original description of the 4-phase algorithm
   - Group-theoretic foundation

2. **Korf, R. E. (1997)** - "Finding Optimal Solutions to Rubik's Cube Using Pattern Databases"
   - Analysis of Thistlethwaite vs optimal solving
   - Pattern database techniques

3. **Rokicki, T., et al. (2010)** - "God's Number is 20"
   - Proof that 20 moves is maximum
   - Shows Thistlethwaite is sub-optimal

### Implementation Resources

- Project source: `src/thistlethwaite/`
- Tests: `tests/thistlethwaite/`
- Benchmarks: `results/phase8_comprehensive_benchmark.json`

### Additional Reading

- Cube group theory: [Wikipedia - Rubik's Cube group](https://en.wikipedia.org/wiki/Rubik%27s_Cube_group)
- Algorithm comparison: Notebook 05_Algorithm_Comparison.ipynb
- Kociemba's improvement: Notebook 03_Kociemba.ipynb

## Conclusion

Thistlethwaite's algorithm was revolutionary for its time, being the first algorithm to solve the cube in under a minute (vs hours for earlier methods).

### Key Takeaways

1. **Group theory is powerful** - Breaking problem into nested subgroups
2. **Trade-offs matter** - Fast but sub-optimal solutions
3. **Foundation for improvements** - Led to Kociemba's 2-phase algorithm
4. **Still relevant** - Great for teaching and understanding

### Next Steps

- Explore **03_Kociemba.ipynb** - See how Kociemba improved on this
- Explore **04_Korf.ipynb** - Learn about optimal solving
- Explore **05_Algorithm_Comparison.ipynb** - Compare all three

---

**Happy Cubing! 🎲**